# Solubility of uranium for changing pH

<p class="acknowledgement">Written jointly by Svetlana Kyas (ETH Zurich) and Dan Miron (PSI) on April 4th, 2022</p>

This tutorial shows an example of the calculation of uranium solubility at changing pH values in groundwater. This example uses the thermodynamic database `psinagra-12-07`, which is based on the [Nagra/PSI Chemical Thermodynamic Data Base 01/01] (https://www.psi.ch/de/les/database). It supports the ongoing safety assessments for the planned low and intermediate-level (L/ILW) and high-level (HLW) radioactive waste repositories in Switzerland.

```{note}
If your main interest is in calculating thermodynamic properties rather than modeling chemical equilibria and kinetics, you should check out [ThermoFun] (https://thermohub.org/thermofun/thermofun/), an excellent project dedicated to this task.
```

First, we set up the chemical system with the aqueous phase only.

In [1]:
from reaktoro import *
import numpy as np

# Define the Thermofun database
db = ThermoFunDatabase ("psinagra-12-07")

# Define the aqueous phase
solution = AqueousPhase(speciate("C Cl H O P S U"))
solution.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2")
))

# Define chemical system by providing database and an aqueous phase
system = ChemicalSystem(db, solution)
species_with_u = [species.name() for species in system.species() if "U" in species.name()]
for species in species_with_u:
    print(species)
# "UO2+2 UO2OH+ UO2(OH)2@ UO2CO3@ (UO2)3CO3(OH)3+ (UO2)2CO3(OH)3- (UO2)2(OH)+3 " \
# "UO2(CO3)3-4 UO2(CO3)2-2 (UO2)2(OH)2+2 (UO2)3(OH)5+ (UO2)4(OH)7+"

(UO2)2(OH)+3
(UO2)2(OH)2+2
(UO2)2CO3(OH)3-
(UO2)3(CO3)6-6
(UO2)3(OH)4+2
(UO2)3(OH)5+
(UO2)3(OH)7-
(UO2)3CO3(OH)3+
(UO2)4(OH)7+
U(CO3)4-4
U(CO3)5-6
U(OH)2+2
U(OH)3+
U(OH)4@
U(SO4)+2
U(SO4)2@
U+4
UCO3(OH)3-
UCl+3
UO2(CO3)2-2
UO2(CO3)3-4
UO2(CO3)3-5
UO2(H2PO4)+
UO2(H2PO4)2@
UO2(H3PO4)+2
UO2(HPO4)@
UO2(OH)2@
UO2(OH)3-
UO2(OH)4-2
UO2(PO4)-
UO2(SO4)2-2
UO2(SO4)3-4
UO2(SO4)@
UO2+
UO2+2
UO2CO3@
UO2Cl+
UO2Cl2@
UO2H5(PO4)2+
UO2OH+
UOH+3


The equilibrium specifications given below indicate that the equilibrium calculations are performed for fixed T, P, pH, and partial pressure of CO<sub>2</sub>. The corresponding equilibrium conditions and the equilibrium solver are initialized with the instance `specs`.

In [2]:
# Specify conditions to be satisfied at chemical equilibrium
specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.pH()
specs.fugacity("CO2(g)")

# Define conditions to be satisfied at the chemical equilibrium state
conditions = EquilibriumConditions(specs)
conditions.temperature(25.0, "celsius")
conditions.pressure(1.0, "bar")
conditions.fugacity("CO2(g)", 0.01, "bar")

# Define the equilibrium solver
solver = EquilibriumSolver(specs)

The initial chemical state is defined according to the following recipe:

* 1000 g of water and
* 1e-5 mol of UO<sub>2</sub>(OH)<sub>2</sub>.

In [3]:
# Define initial equilibrium state
state = ChemicalState(system)
state.set("H2O@"     , 1e3,  "g")
state.set("UO2(OH)2@", 1e-5, "mol")

The amount of uranium can be calculated based on the chemical properties of this chemical state. We define the range of pH values and the list of uranium-containing species of interest for our evaluation.

In [15]:
# Calculate the amount of uranium element
props = ChemicalProps(state)

# Defined an auxiliary array of pH values
pHs = np.linspace(3, 10, num=71)
print(pHs)

# Create list of species names, list of Species objects, and auxiliary amounts array
species_with_u = [species.name() for species in system.species() if "U" in species.name()]

#species_list_str = "UO2+2 UO2OH+ UO2(OH)2@ UO2CO3@ (UO2)3CO3(OH)3+ (UO2)2CO3(OH)3- (UO2)2(OH)+3 " \
#                   "UO2(CO3)3-4 UO2(CO3)2-2 (UO2)2(OH)2+2 (UO2)3(OH)5+ (UO2)4(OH)7+".split()
species_list_str = species_with_u
percentages = np.zeros(len(species_list_str))
amounts = np.zeros(len(species_list_str))

# Define dataframe to collect amount of the selected species
import pandas as pd
columns = ["pH"] \
          + ["amount_" + name for name in species_list_str] \
          + ["perc_" + name for name in species_list_str]
df = pd.DataFrame(columns=columns)

[ 3.   3.1  3.2  3.3  3.4  3.5  3.6  3.7  3.8  3.9  4.   4.1  4.2  4.3
  4.4  4.5  4.6  4.7  4.8  4.9  5.   5.1  5.2  5.3  5.4  5.5  5.6  5.7
  5.8  5.9  6.   6.1  6.2  6.3  6.4  6.5  6.6  6.7  6.8  6.9  7.   7.1
  7.2  7.3  7.4  7.5  7.6  7.7  7.8  7.9  8.   8.1  8.2  8.3  8.4  8.5
  8.6  8.7  8.8  8.9  9.   9.1  9.2  9.3  9.4  9.5  9.6  9.7  9.8  9.9
 10. ]


Finally, we run simulations in the for loop on the pH values provided for the instance of the equilibrium conditions.

In [16]:
for pH in pHs:

    # Set the value of pH for the current equilibrium calculations
    conditions.pH(pH)

    # Equilibrate the initial state with given conditions and component amounts
    res = solver.solve(state, conditions)

    props.update(state)
    #bU = props.elementAmount("U")[0]
    bU = 1e-5
    
    if pH == 6: 
        print(props)

    # If the equilibrium calculations didn't succeed, continue to the next condition
    if not res.optima.succeeded: continue

    # Otherwise, calculate U(VI) speciation in mols and %
    for j in range(0, len(species_list_str)):
        amounts[j] = float(state.speciesAmount(species_list_str[j]))
        percentages[j] = float(state.speciesAmount(species_list_str[j])) / bU * 100
        
    # Update dataframe with obtained values
    df.loc[len(df)] = np.concatenate([[pH], amounts, percentages])

+------------------------------------------+----------------+-----------+
| Property                                 |          Value |      Unit |
+------------------------------------------+----------------+-----------+
| Temperature                              |       298.1500 |         K |
| Pressure                                 |         1.0000 |       bar |
| Volume                                   |     1.0030e-03 |        m3 |
| Gibbs Energy                             | -13165772.5873 |         J |
| Enthalpy                                 | -15866234.3254 |         J |
| Entropy                                  |     -9057.3931 |       J/K |
| Internal Energy                          | -15866334.6226 |         J |
| Helmholtz Energy                         | -13165872.8845 |         J |
| Charge                                   |    -1.5478e-04 |       mol |
| Element Amount:                          |                |           |
| :: H                                

In the following, we plot the obtained values of U(VI) speciation in % (using [bokeh](https://bokeh.org/) plotting library).

In [17]:
df6 = df[df["pH"] == 6]

In [18]:
df6

,pH,amount_(UO2)2(OH)+3,amount_(UO2)2(OH)2+2,amount_(UO2)2CO3(OH)3-,amount_(UO2)3(CO3)6-6,amount_(UO2)3(OH)4+2,amount_(UO2)3(OH)5+,amount_(UO2)3(OH)7-,amount_(UO2)3CO3(OH)3+,amount_(UO2)4(OH)7+,...,perc_UO2(SO4)3-4,perc_UO2(SO4)@,perc_UO2+,perc_UO2+2,perc_UO2CO3@,perc_UO2Cl+,perc_UO2Cl2@,perc_UO2H5(PO4)2+,perc_UO2OH+,perc_UOH+3
30,6.0,6.555760e-12,7.484898e-09,0.000003,1.000000e-16,2.149213e-10,4.664370e-08,1.043990e-12,5.373140e-12,1.139952e-09,...,1.000000e-09,1.000000e-09,1.000000e-09,0.570281,33.846861,1.000000e-09,1.000000e-09,1.000000e-09,3.108803,1.000000e-09


In [19]:
species = ["pH"] + species_list_str + ["perc_" + name for name in species_list_str]
print(species)
total = 0
total_amount = 0
for species, elem in zip(species, df6.loc[30]):
    #if "perc_" in species and elem > 0.5:
    print(f"{species} : {elem}")
    if "perc_" in species:
        total += elem
    if "perc_" not in species:
        total_amount += elem
print(total)
print(total_amount - 6)

['pH', '(UO2)2(OH)+3', '(UO2)2(OH)2+2', '(UO2)2CO3(OH)3-', '(UO2)3(CO3)6-6', '(UO2)3(OH)4+2', '(UO2)3(OH)5+', '(UO2)3(OH)7-', '(UO2)3CO3(OH)3+', '(UO2)4(OH)7+', 'U(CO3)4-4', 'U(CO3)5-6', 'U(OH)2+2', 'U(OH)3+', 'U(OH)4@', 'U(SO4)+2', 'U(SO4)2@', 'U+4', 'UCO3(OH)3-', 'UCl+3', 'UO2(CO3)2-2', 'UO2(CO3)3-4', 'UO2(CO3)3-5', 'UO2(H2PO4)+', 'UO2(H2PO4)2@', 'UO2(H3PO4)+2', 'UO2(HPO4)@', 'UO2(OH)2@', 'UO2(OH)3-', 'UO2(OH)4-2', 'UO2(PO4)-', 'UO2(SO4)2-2', 'UO2(SO4)3-4', 'UO2(SO4)@', 'UO2+', 'UO2+2', 'UO2CO3@', 'UO2Cl+', 'UO2Cl2@', 'UO2H5(PO4)2+', 'UO2OH+', 'UOH+3', 'perc_(UO2)2(OH)+3', 'perc_(UO2)2(OH)2+2', 'perc_(UO2)2CO3(OH)3-', 'perc_(UO2)3(CO3)6-6', 'perc_(UO2)3(OH)4+2', 'perc_(UO2)3(OH)5+', 'perc_(UO2)3(OH)7-', 'perc_(UO2)3CO3(OH)3+', 'perc_(UO2)4(OH)7+', 'perc_U(CO3)4-4', 'perc_U(CO3)5-6', 'perc_U(OH)2+2', 'perc_U(OH)3+', 'perc_U(OH)4@', 'perc_U(SO4)+2', 'perc_U(SO4)2@', 'perc_U+4', 'perc_UCO3(OH)3-', 'perc_UCl+3', 'perc_UO2(CO3)2-2', 'perc_UO2(CO3)3-4', 'perc_UO2(CO3)3-5', 'perc_UO2(H2PO4)

In [9]:
from bokeh.plotting import figure, show
from bokeh.models import Range1d
from bokeh.io import output_notebook
output_notebook()

p = figure(
    title="",
    x_axis_label=r'PH [-]',
    y_axis_label='U(VI) SPECIATION [%]',
    sizing_mode="scale_width",
    plot_height=300)

colors = ['teal', 'darkred', 'indigo', 'coral', 'rosybrown', 'steelblue', 'seagreen', 'palevioletred', 'darkred', 'darkkhaki', 'cadetblue', 'indianred']
for species, color in zip(species_list_str, colors):
    p.line("pH", "perc_"+species, legend_label=species, line_width=3, line_cap="round", line_color=color, source=df)
p.legend.location = 'center_right'

show(p)

p1 = figure(
    title="",
    x_axis_label=r'PH [-]',
    y_axis_label='log(U(VI)) SPECIATION [MOL]',
    y_axis_type = 'log',
    sizing_mode="scale_width",
    plot_height=300)

colors = ['teal', 'darkred', 'indigo', 'coral', 'rosybrown', 'steelblue', 'seagreen', 'palevioletred', 'darkred', 'darkkhaki', 'cadetblue', 'indianred']
for species, color in zip(species_list_str, colors):
    p1.line("pH", "amount_"+species, legend_label=species, line_width=3, line_cap="round", line_color=color, source=df)
p1.legend.location = 'center_right'

show(p1)

Loading BokehJS ...

Note, that for the acidic solution, species such as UO2<sup>+2</sup>, UO<sub>2</sub>OH<sup>+</sup>, UO<sub>2</sub>CO<sub>3</sub> are predominantly present. For the alkaline solution, UO(CO<sub>3</sub>)<sub>3</sub><sup>-4</sup> are growing rapidly taking almost 100% of the aqueuos phase. For pH between 6 and 8, UO<sub>2</sub>(CO3)<sub>2</sub><sup>-2</sup> grows the highests reaching its maximum for pH = 7.2.